<a href="https://colab.research.google.com/github/FisherCh/pytorch-tutorial/blob/master/sample_deepspeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# --------------------- 
# 作者：LeoWood 
# 来源：CSDN 
# 原文：https://blog.csdn.net/u014108004/article/details/84325301 
# 版权声明：本文为博主原创文章，转载请附上博文链接！

In [0]:
# 挂载谷歌云端硬盘
!mkdir -p drive
!google-drive-ocamlfuse drive  -o nonempty

# requirements
 - pytorch v1.0
 - warp-ctc
 - pytorch-audio
 - deepspeech

In [0]:
!pip install python_Levenshtein

    100% |████████████████████████████████| 51kB 2.1MB/s 
  Running setup.py bdist_wheel for python-Levenshtein ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install torch torchvision

# !conda install pytorch torchvision -c pytorch

import torch
print(torch.__version__)

    100% |████████████████████████████████| 591.8MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x612e8000 @  0x7f81094d42a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 25.1MB/s 
    100% |████████████████████████████████| 2.0MB 7.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
1.0.0


In [0]:
# !apt-get install sox libsox-dev libsox-fmt-all
# !git clone https://github.com/pytorch/audio.git
# !pip install cffi
# !python audio/setup.py install
!pip install git+git://github.com/pytorch/audio
!ls

  Cloning git://github.com/pytorch/audio to /tmp/pip-req-build-w83719pz
  Running setup.py bdist_wheel for torchaudio ... - \ | / - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-ot2su_4y/wheels/4e/7c/c5/0d946acbaccad9fe62590374454c4cf135846c9c96fce3ac75
Successfully built torchaudio
audio  build  sample_data  torchaudio.egg-info


In [0]:
# example of audio
import torchaudio
sound, sample_rate = torchaudio.load('foo.mp3')
torchaudio.save('foo_save.mp3', sound, sample_rate)

OSError: ignored

In [0]:
# Install baidu wrap-ctc
# !git clone https://github.com/SeanNaren/warp-ctc.git
# !pwd
# !ls
# !cd warp-ctc;mkdir build; cd build; pwd; cmake ..; make
!cd warp-ctc; cd pytorch_binding; python setup.py install

running install
running bdist_egg
running egg_info
creating warpctc_pytorch.egg-info
writing warpctc_pytorch.egg-info/PKG-INFO
writing dependency_links to warpctc_pytorch.egg-info/dependency_links.txt
writing top-level names to warpctc_pytorch.egg-info/top_level.txt
writing manifest file 'warpctc_pytorch.egg-info/SOURCES.txt'
reading manifest file 'warpctc_pytorch.egg-info/SOURCES.txt'
writing manifest file 'warpctc_pytorch.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/warpctc_pytorch
copying warpctc_pytorch/__init__.py -> build/lib.linux-x86_64-3.6/warpctc_pytorch
running build_ext
building 'warpctc_pytorch._warp_ctc' extension
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-t

In [0]:
# example for ctc loss
import torch
from warpctc_pytorch import CTCLoss
ctc_loss = CTCLoss()
# expected shape of seqLength x batchSize x alphabet_size
probs = torch.FloatTensor([[[0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]]]).transpose(0, 1).contiguous()
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)  # tells autograd to compute gradients for probs
cost = ctc_loss(probs, labels, probs_sizes, label_sizes)
cost.backward()

ModuleNotFoundError: ignored

In [0]:
!git clone https://github.com/SeanNaren/deepspeech.pytorch.git
!ls

Cloning into 'deepspeech.pytorch'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1199 (delta 1), reused 1 (delta 0), pack-reused 1188
Receiving objects: 100% (1199/1199), 373.37 KiB | 688.00 KiB/s, done.
Resolving deltas: 100% (752/752), done.
audio  build  deepspeech.pytorch  sample_data  torchaudio.egg-info  warp-ctc


In [0]:
# !cd deepspeech.pytorch; python transcribe.py --model_path models/deepspeech.pth.tar --audio_path /path/to/audio.wav
!cd deepspeech.pytorch; python transcribe.py --audio-path /../001.wav

Traceback (most recent call last):
  File "transcribe.py", line 67, in <module>
    model = DeepSpeech.load_model(args.model_path)
  File "/content/deepspeech.pytorch/model.py", line 239, in load_model
    package = torch.load(path, map_location=lambda storage, loc: storage)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 365, in load
    f = open(f, 'rb')
FileNotFoundError: [Errno 2] No such file or directory: 'models/deepspeech_final.pth'


# 新段落

In [0]:
import os
os.getcwd()

'/content'

In [0]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


In [0]:
# import torch
import torch.nn as nn
from torch.nn import DataParallel
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

print(torch.__version__)


1.0.0


In [0]:
import numpy as np
import random
class constants(object):
    def __init__(self):
        self.PAD = 0
        self.UNK = 1
        self.BOS = 2
        self.EOS = 3

        self.PAD_WORD = '<pad>'
        self.UNK_WORD = '<unk>'
        self.BOS_WORD = '<s>'
        self.EOS_WORD = '</s>'
Constants = constants()

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers=2, dropout=0.2):
        super().__init__()

        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        # input_dim = vocab_size + 4
        # self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.GRU(embedding_dim, hidden_dim,
                          num_layers=num_layers, bidirectional=True)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input_seqs, input_lengths, embedding, hidden=None):
        # src = [sent len, batch size]
        embedded = embedding(input_seqs)
        # embedded = [sent len x batch size x emb dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.rnn(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        
        del embedded, packed, output_lengths
        
        outputs = outputs[:,:,:self.hidden_dim] + outputs[:,:,self.hidden_dim:]
        hidden = hidden[-1,:,:] + hidden[-2,:,:]
        # outputs = [sent len, batch size, hid dim * n directions]
        # hidden = [n layers, batch size, hid dim]
        # outputs are always from the last layer
        return outputs, hidden.unsqueeze(0)

In [0]:
def attention(query, key, value=None):
    """query[B, H], key[B, L, H], value[B, L, H]"""
    value = key
    query = query.unsqueeze(1).repeat(1, key.size(1), 1)
    # (B, L, H)
    score = torch.sum(query * key, -1)
    attn = score.softmax(-1).unsqueeze(1)
    del score
    # (B, 1, L)
    outputs = torch.matmul(attn, value)
    return outputs.squeeze(1), attn.squeeze(1)

class DecoderWithAttn(nn.Module):
    def __init__(self, voc_siz, emb_siz, hid_siz, num_layers=1, dropout=0.2):
        super().__init__()

        self.emb_siz = emb_siz
        self.hid_siz = hid_siz
        self.voc_siz = voc_siz
        self.num_layers = num_layers

        self.rnn = nn.GRU(emb_siz, hid_siz,
                          num_layers=num_layers)
        self.out = nn.Linear(hid_siz * 2, voc_siz)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inp, h, embedding, enc_outputs):
        # input = [bsz]
        # hidden = [n layers * n directions, batch size, hid dim]
        # encoder_outputs = [sent len, batch size, hid dim * n directions]
        # input : [bsz] -> [1, bsz]
        embedded = embedding(inp.unsqueeze(0))
        # embeded = [1, bsz, emb dim]
        _, hidden = self.rnn(embedded, h)
        
        # attn
        # context: [bsz, hid], attn: [bsz, seq]
        context, attn = attention(hidden[-1], enc_outputs.permute(1,0,2), value=None)
        
        emb_con = torch.cat([hidden[-1], context], dim=-1)
        
        output = self.out(self.dropout(emb_con)).log_softmax(-1)
        # outputs = [sent len, batch size, vocab_size]
        return output, hidden, attn

In [0]:
# example

encoder = Encoder(input_dim=10, embedding_dim=20, hidden_dim=10)
decoder = DecoderWithAttn(voc_siz=10, emb_siz=20, hid_siz=10)

embedding = nn.Embedding(10, 20)

inp = (torch.rand(2, 3).relu() * 4).long()
# inp = Variable(inp, requires_grad=False)
print(inp)

outputs, h = encoder(inp.permute(1, 0), np.array([3, 3]), embedding)

decoder(inp[:, 0], h, embedding, outputs)

tensor([[1, 0, 1],
        [1, 1, 3]])


(tensor([[-2.4960, -2.0804, -2.3292, -2.1563, -2.8775, -2.1951, -2.0818, -2.6549,
          -2.1324, -2.3188],
         [-2.6771, -1.9753, -2.2301, -2.1704, -2.8830, -2.1328, -2.2388, -2.5547,
          -2.2098, -2.2792]], grad_fn=<LogSoftmaxBackward>),
 tensor([[[ 0.2681, -0.4831,  0.2203,  0.1389, -0.1867, -0.1735,  0.2489,
            0.0462, -0.4146, -0.0252],
          [ 0.1834, -0.5881,  0.2708, -0.0189, -0.1866, -0.1978,  0.2123,
           -0.0466, -0.3120, -0.0733]]], grad_fn=<ViewBackward>),
 tensor([[0.3027, 0.3448, 0.3525],
         [0.3294, 0.3573, 0.3133]], grad_fn=<SqueezeBackward1>))

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, word_emb, encoder, decoder, teacher_forcing_ratio=1):
        super().__init__()
        
        global Constants
        
        self.word_emb = word_emb
        self.encoder = encoder
        self.decoder = decoder
        
        #self.cos = nn.CosineSimilarity(dim=2, eps=1e-6)
        self.cos = nn.CosineEmbeddingLoss()
        self.nll = nn.NLLLoss(ignore_index=Constants.PAD)
        
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.lambda_ = 0.1
        
        
    def forward(self, inp, src_lengths, mode=True):
        global config, Constants, device
        
        enc_outputs, h = self.encoder(inp.permute(1, 0), src_lengths, self.word_emb)
        
        if mode:
            # auto encode
            loss, output = self.nllLoss(h, enc_outputs, target=inp)
        else:
            # abstract
            loss, output = self.cosLoss(h, enc_outputs, target=inp)
        
        return loss, output
        
    def decode(self, h, enc_outputs, target, teacher_rate=1, use_mask=False):
        """
        Input:
         - h:       1 x batch_siz x hid_siz, tensor
         - enc_outputs: seq_len x batch_siz x hid_siz
         - target:  batch_siz x seq_len, tensor
        Output:
         - outputs: seq_len x batch_siz x voc_siz, tensor
        """
        global config, Constants, device
        
        batch_size = h.size(1)
        num_word = config['siz_voc']
        max_len = target.size(1)
        
        # tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, num_word).to(device)
        outputs[1:, :, Constants.PAD] = 1
        outputs[0, :, Constants.BOS] = 1                                    # seq_len x batch x vocab
        outputs = outputs.log_softmax(-1)
        
        # end token
        end_token = torch.zeros(batch_size)
        # Coverage mechanism
        c_t = torch.zeros(batch_size, enc_outputs.size(0)).to(device)
        loss_cov = 0
        # decoder
        tar = target.permute(1, 0)
        out = torch.ones(batch_size).long().to(device) * Constants.BOS
        for t in range(1, max_len):                                        # skip sos
            out, h, attn = self.decoder(out, h, self.word_emb, enc_outputs)
            
            loss_cov = loss_cov + torch.min(c_t, attn).sum()
            c_t = c_t + attn
            
            outputs[t] = out
            
            # mask end token
            if use_mask:
                # record end token
                out_copy = out.cpu().detach().max(-1)[1]
                end_token += (out_copy == Constants.EOS).squeeze().type(end_token.type()) + \
                             (out_copy == Constants.PAD).squeeze().type(end_token.type())
                if (end_token>0).sum() >= end_token.size(0):
                    break
                # mask
                outputs[t, end_token>0, :] = 0
                outputs[t, end_token>0, Constants.PAD] = 1
                outputs[t, end_token>0, :] = outputs[t, end_token>0, :].log_softmax(-1)
            
            # next word
            teacher_force = random.random() < teacher_rate
            out = (tar[t] if teacher_force else out.max(-1)[1])
        
        del h, out, tar
        
        return outputs, loss_cov/t
    
    def nllLoss(self, h, enc_outputs, target):
        
        outputs, loss_cov = self.decode(h, enc_outputs, target, 
                                        teacher_rate=self.teacher_forcing_ratio)        # seq x batch(1) x voc
        
        loss = self.nll(
            outputs[1:].contiguous().view(-1, outputs.size(-1)), 
            target.permute(1, 0)[1:].contiguous().view(-1))

        return loss + self.lambda_ * loss_cov, outputs

    def cosLoss(self, h, enc_outputs, target):
        """
        Input:
         - h: 1 x batch x hid
        """
        batch_siz = h.size(1)
        max_len = target.size(1)
        
        enc_mean = enc_outputs.mean(1).unsqueeze(1)                                     # seq x 1 x hid
        h_mean = h.mean(1).unsqueeze(1)                                                 # 1 x 1 x hid
        
        outputs, loss_cov = self.decode(h_mean, enc_mean, target[:1,:], teacher_rate=0)           # seq x batch(1) x voc
        _, h_hat = self.encoder(
            outputs.argmax(-1), 
            np.array([max_len] * outputs.size(1)), 
            self.word_emb) # 1 x 1 x hid
        
        h_hat = h_hat.squeeze(0)
        y = torch.ones(batch_siz).to(device)
        sim = self.cos(h_hat, h.squeeze(0), y)
        
        del h_hat, h_mean
        return sim, outputs
    

In [0]:
# example

device = torch.device("cpu")

config = {
    'use_cuda': True,
    
    'siz_voc': 10,  # word number
    'siz_emb': 20,                     # embedding dim of word & tag
    'siz_hid': 10,                     # hidden size of GRU
    
    'seq_len': 5,                      # max sequence length
    'siz_batch': 64,                  # batch size of Seq2seq
    'step_seq': 10000,
    
}


encoder = Encoder(config["siz_voc"], config["siz_emb"], config["siz_hid"]).to(device)
decoder = DecoderWithAttn(config["siz_voc"], config["siz_emb"], config["siz_hid"]).to(device)
embedding = nn.Embedding(config["siz_voc"], config["siz_emb"]).to(device)

model = Seq2Seq(embedding, encoder, decoder).to(device)


inp = torch.randint(10, (2, 5)).long()
# inp = Variable(inp, requires_grad=False)
print(inp)

nllloss, out_AE = model(inp, np.array([5, 4]), mode=True)
cosloss, out_Avg = model(inp, np.array([5, 4]), mode=False)

nllloss.backward()

print("NLLLoss: ", nllloss)
print("cosLoss: ", cosloss)

print("AE: ", out_AE.size())
print("abstract: ", out_Avg.size())

tensor([[0, 4, 6, 0, 7],
        [5, 5, 4, 7, 2]])
NLLLoss:  tensor(2.6199, grad_fn=<ThAddBackward>)
cosLoss:  tensor(0.1302, grad_fn=<DivBackward0>)
AE:  torch.Size([5, 2, 10])
abstract:  torch.Size([5, 1, 10])


# data


In [0]:
from numpy import linalg as LA
import pickle as pickle
import random
from collections import Counter

class Vocabulary(object):
    def __init__(self, vocab_file, emb_file='', dim_emb=0):
        with open(vocab_file, 'rb') as f:
            self.size, self.word2id, self.id2word = pickle.load(f)
        self.dim_emb = dim_emb
        self.embedding = np.random.random_sample(
            (self.size, self.dim_emb)) - 0.5

        if emb_file:
            print('Loading word vectors from', emb_file)
            with open(emb_file) as f:
                for line in f:
                    parts = line.split()
                    word = parts[0]
                    vec = np.array([float(x) for x in parts[1:]])
                    if word in self.word2id:
                        self.embedding[self.word2id[word]] = vec

        for i in range(self.size):
            self.embedding[i] /= LA.norm(self.embedding[i])

def build_vocab(data, path, min_occur=5):
    word2id = {'<pad>':0, '<go>':1, '<eos>':2, '<unk>':3}
    id2word = ['<pad>', '<go>', '<eos>', '<unk>']

    words = [word for sent in data for word in sent]
    cnt = Counter(words)
    for word in cnt:
        if cnt[word] >= min_occur:
            word2id[word] = len(word2id)
            id2word.append(word)
    vocab_size = len(word2id)
    with open(path, 'wb') as f:
        pickle.dump((vocab_size, word2id, id2word), f, pickle.HIGHEST_PROTOCOL)

In [0]:
num_max = 20000
train0 = load_sent(args.train + '.0', num_max)
train1 = load_sent(args.train + '.1', num_max)